In [ ]:
import os
import datetime as dt
import time
import pytz
import pandas as pd

from uce_resources import get_site_id, get_mms_data

In [ ]:
from settings.sites import ceg as sites_list

target_year = 2022
target_month = 4

target_folder = 'data/operation_report/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

sites_list = ['Pohrebyshche', 'Bilashky', 
                'Bar', 'Balky', 'Verkhivka', 'Kopaihorod', 'Stanislavchyk', 'Sharhorod_1', 'Hnatkiv', 'Porohy',
                'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Cherniatka','Chechelnyk_1', 'Chechelnyk_2',
                'Komyshany_1', 'Komyshany_2', 'Bilozerka', 'Myroliubivka', 'Kyselivka',
                'Oleshky_1', 'Oleshky_2', 'Poniativka', 'Kostohryzove', 
                'Veliton', 'Velihen', 
                'Mykolaivka', 'Rubanivka', 'Mala_Lepetykha', 'Kachkarivka', 
                'Liubymivka', 'Vasylivka',
                'Dibrovka', 'Yelanets_1', 'Yelanets_2', 
                'Afanasiivka', 'Novokondakove', 'Bazaltova', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 
                'Teplychna', 
                'Balivka', 'Solone', 'Stepnohirsk', 
                'Kulevcha']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
with engine.connect() as connection:

    yield_list = list()
    consumption_list = list()

    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        mms=pd.DataFrame()
   

        mms_data, site_data['mms_version'] = get_mms_data(site_data['site_id'], 
                                                        target_year, target_month, 
                                                        connection, metadata.tables['mms_data'], include_prev=False)
        mms_data.index = mms_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev'))
        # print(mms_data)
        mms_data_yield = mms_data.loc[mms_data['yield [kWh]'] >= 0].resample('1D').sum()
        mms_data_consumption = mms_data.loc[mms_data['yield [kWh]'] < 0].resample('1D').sum()

        mms_data_yield.index = mms_data_yield.index.date
        mms_data_consumption.index = mms_data_consumption.index.date

        mms_data_yield = mms_data_yield.T
        mms_data_consumption = mms_data_consumption.T
        mms_data_yield.index.name = 'site'
        mms_data_yield.index = [site]
        mms_data_consumption.index = [site]

        yield_list.append(mms_data_yield)
        consumption_list.append(mms_data_consumption)
        print('Ok!')
    
    generation = pd.concat(yield_list)
    consumption = pd.concat(consumption_list)

    generation.to_excel(target_folder + f'generation_{target_year}_{target_month}.xlsx')
    consumption.to_excel(target_folder + f'consumption_{target_year}_{target_month}.xlsx')

    # print(generation)